In [1]:
import re
import os
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
from pyspark.sql import Window
from functools import reduce

# CONSTANTS
HOST = 'output-db'
PORT = 5433
DATABASE = 'postgres'
USER = 'postgres'
PASSWORD = '1234'

INPUT_PATH = './notebooks/parquet/'
TABLE_NAMES = [
    'city',
    'inventory',
    'invoice',
    'product',
    'purchase',
    'purchase_order',
    'sale',
    'store',
    'vendor',
]

spark= (
    SparkSession.builder.appName("write-to-db")
    .config("spark.jars", "/usr/local/spark/jars/postgresql.jar")    
    .getOrCreate()
)

_df_raw={}
for table_name in TABLE_NAMES:
    filename=rf'{INPUT_PATH}/{table_name}'
    _df_raw[table_name]=spark.read.format('parquet').option("header",True).load(filename)
    jdbc_url = f'jdbc:postgresql://{HOST}:{PORT}/{DATABASE}'
    properties = {
        'user': f'{USER}',
        'password': f'{PASSWORD}',
        'driver': 'org.postgresql.Driver'
    }

    print(f'Writing {table_name} to PostgreSQL database...')
    _df_raw[table_name].write.jdbc(url=jdbc_url, table=table_name, mode="overwrite", properties=properties)
print('Finished!')


Writing city to PostgreSQL database...
Writing inventory to PostgreSQL database...
Writing invoice to PostgreSQL database...
Writing product to PostgreSQL database...
Writing purchase to PostgreSQL database...
Writing purchase_order to PostgreSQL database...
Writing sale to PostgreSQL database...
Writing store to PostgreSQL database...
Writing vendor to PostgreSQL database...
Finished!


In [2]:
# TODO: use psycopg2 to add PRIMARY KEY constraints